In [1]:
import pandas as pd

# 指定源文件和目標文件路徑
input_file = '/Users/chenmj/Documents/GitHub/python_dash-flask/data2/2024.csv'
output_file = '/Users/chenmj/Documents/GitHub/python_dash-flask/data2/2024_utf8.csv'

# 使用pandas讀取UTF-16編碼的CSV文件
df = pd.read_csv(input_file, encoding='utf-16')

# 將DataFrame寫入UTF-8編碼的CSV文件
df.to_csv(output_file, encoding='utf-8', index=False)

print(f"文件已成功從UTF-16轉換為UTF-8並保存為: {output_file}")

文件已成功從UTF-16轉換為UTF-8並保存為: /Users/chenmj/Documents/GitHub/python_dash-flask/data2/2024_utf8.csv


In [2]:
import psycopg2
import pandas as pd
from psycopg2.extras import execute_values

# 连接到数据库

dbname="python"
user="tvdi_postgresql_new_b3wx_user"
password="K4XGCCwSgJ6jWdBNhTLq9JniYjzmvIH2"
host="dpg-cqa8dr5ds78s739nis2g-a.singapore-postgres.render.com"
port="5432"

try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port,
        sslmode='require'  # 启用 SSL/TLS 连接
    )
    print("Connection successful")
except psycopg2.OperationalError as e:
    print(f"Connection failed: {e.pgcode} - {e.pgerror}")
    raise

cur = conn.cursor()

# 读取CSV文件
df = pd.read_csv('/Users/chenmj/Documents/GitHub/python_dash-flask/data2/2024_utf8.csv')

# 插入数据
try:
    # 将DataFrame转换为list of tuples
    data_tuples = [tuple(x) for x in df.to_numpy()]
    columns = list(df.columns)
    
    # 使用execute_values进行批量插入
    insert_query = f"""
        INSERT INTO public.traffic2024 ({', '.join(['"{}"'.format(col) for col in columns])})
        VALUES %s
    """
    execute_values(cur, insert_query, data_tuples)
    conn.commit()
    print("Data inserted successfully")
except Exception as e:
    print(f"Data insertion failed: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

Connection successful
Data inserted successfully
